In [30]:
import pandas as pd
from tqdm import tqdm
from kiwipiepy import Kiwi
from keybert import KeyBERT
from transformers import BertModel

# 모델 초기화
model = BertModel.from_pretrained('skt/kobert-base-v1')  # 한국어 BERT 모델을 로드합니다.
kw_model = KeyBERT(model)  # KeyBERT 모델을 초기화합니다. 이 모델은 키워드 추출에 사용됩니다.
kiwi = Kiwi()  # Kiwi 한국어 형태소 분석기를 초기화합니다.

# Load the CSV file again
data = pd.read_csv('preprocessed_total_reviews_market.csv')

# Extracting each row from the 'preprocessed_content' column and storing it in a list
content_list = data['preprocessed_content'].tolist()

In [31]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)  # 텍스트를 분석하여 형태소 분석 결과를 얻습니다.
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):  # 명사와 외래어를 추출합니다.
            results.append(token)
    return results

# 키워드 추출
extracted_keywords = []
for text in tqdm(data['content']):  # 데이터의 'content' 열을 순회하며 키워드를 추출합니다.
    # 명사 추출
    nouns = noun_extractor(text)
    after_kiwi_text = ' '.join(nouns)
    
    # KeyBERT를 사용하여 키워드 추출
    keywords = kw_model.extract_keywords(after_kiwi_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=3)
    extracted_keywords.append(keywords)

# 추출된 키워드에서 단어만 추출
words_only = [[keyword[0] for keyword in keywords_list] for keywords_list in extracted_keywords]

# 'preprocessed_category'와 'preprocessed_productname' 열을 합쳐서 새로운 열 생성
data['Extracted_Keywords'] = data['preprocessed_category'] + " " + data['preprocessed_productname']

# 기존 'hashtags' 열 유지
# 'hashtags' 열은 이전과 같이 추출된 키워드를 해시태그 형태로 저장합니다.
data['hashtags'] = [' '.join(keyword_list) for keyword_list in words_only]

# 나머지 부분은 이전과 동일합니다...

# 새로운 DataFrame 열 생성 및 수정
data.rename(columns={'like': 'importance'}, inplace=True)

# datetime으로 정렬
data['datetime'] = pd.to_datetime(data['datetime'])
data.sort_values(by='datetime', inplace=True)


100%|██████████| 1000/1000 [00:15<00:00, 64.85it/s]


In [32]:
#각 단어 마다 , 추가
def add_commas(string):
    # Split the string by spaces and join with commas
    return ', '.join(string.split())

# commas 함수를 hashtags와 
data['hashtags'] = data['hashtags'].apply(add_commas)
data['Extracted_Keywords'] = data['Extracted_Keywords'].apply(add_commas)

# 처음과 끝에 ' [ '와 ' ] '추가 하는 함수를 hashtags와 Extracted_Keyword에 적용
def add_brackets(string):
    return f'[{string}]'

# 괄호 추가 함수를 hashtags와 Extracted_Keyword에 적용
data['hashtags'] = data['hashtags'].apply(add_brackets)
data['Extracted_Keywords'] = data['Extracted_Keywords'].apply(add_brackets)


data.head()

,category,productname,content,importance,datetime,preprocessed_category,preprocessed_productname,preprocessed_content,Extracted_Keywords,hashtags
412,베이커리·치즈·델리,[교토마블] 데니쉬 식빵 3종 (택1),아무 맛도 안 나는 것 같지만 사실 식빵이야말로 빵의 맛을 좌우한다고 믿는 사람으로...,36,2018-09-23,베이커리치즈델리,교토마블 데니쉬 식빵,아무 맛 도 안 나 는 같지만 사실 식빵 이야말로 빵 맛 좌우 한 다 고 믿는 사람...,"[베이커리치즈델리, 교토마블, 데니쉬, 식빵]","[치즈, 식빵, 피자]"
872,수산·해산·건어물,[Sea to Table] 손질 생새우살 200g (냉동) (26/30 사이즈),여러 종류의 새우가 있는데 그 중에서 손질생새우살은 처음 구입해봤습니다.\n\n냉동...,15,2019-01-26,수산해산건어물,손질 생새우살 냉동 사이즈,여러 종류 새우 가 있는데 중 에서 손질 생 새우 살 은 처음 구입 해봤습니다 냉동...,"[수산해산건어물, 손질, 생새우살, 냉동, 사이즈]","[타면, 신랑, 체다지즈]"
871,수산·해산·건어물,[Sea to Table] 손질 생새우살 200g (냉동) (26/30 사이즈),감바스 알 아히요 만들어 먹었습니다. \n냉동되어 있던 것을 뜨거운물에 담가놓으니 ...,33,2019-02-10,수산해산건어물,손질 생새우살 냉동 사이즈,감 바스 알 아히 요 만들어 먹었습니다 냉동 되어 있던 뜨거운 물 에 담가놓으니 금...,"[수산해산건어물, 손질, 생새우살, 냉동, 사이즈]","[호밀빵, 치즈, 식빵]"
191,과일·견과·쌀,[항공직송] 칠레산 생 블루베리 2종,(market curly 칠레산블루베리(125g)후기입니다)\n\n마켓컬리에서 냉동...,30,2019-02-14,과일견과쌀,항공직송 칠레산 생 블루베리,market curly 칠레 산 블루베리 125 g 후기 입니다 마켓 컬리 에서 냉...,"[과일견과쌀, 항공직송, 칠레산, 생, 블루베리]","[마켓컬리, 푸드, 굿굿]"
870,수산·해산·건어물,[Sea to Table] 손질 생새우살 200g (냉동) (26/30 사이즈),저는 나가서 사먹거나 시켜먹는것보다 해먹는걸 좋아하는 20대 남자주부(?)입니다ㅋㅋ...,140,2019-03-18,수산해산건어물,손질 생새우살 냉동 사이즈,저 는 나가서 사먹거나 시켜 먹는것보다 해먹 는걸 좋아하는 남자 주부 입니다 ㅋㅋㅋ...,"[수산해산건어물, 손질, 생새우살, 냉동, 사이즈]","[마늘, 후추, 페퍼론치노]"


In [37]:
# 특정 열 선택 및 새 CSV 파일로 저장
final_data = data[['datetime', 'hashtags', 'importance', 'Extracted_Keywords']]

# importance가 0 보다 큰 값만 저장
final_data = final_data[final_data['importance'] > 0]

final_csv_path = 'datetime_hashtags_importance_keywords_market.csv'
final_data.to_csv(final_csv_path, index=False)  # 선택된 열을 새 CSV 파일로 저장합니다.

final_csv_path

'C:/Users/kgw57/PycharmProjects/data_contest/keyword_review/datetime_hashtags_importance_keywords_market.csv'